In [1]:
import os
import sys

sys.path.append(os.path.abspath('..'))

In [2]:
import numpy as np
from delay_optimizer.functions import Ackley, Rastrigin, Rosenbrock, Zakharov
from delay_optimizer.optimizers import GradientDescent, Adam, Momentum, NesterovMomentum
from delay_optimizer.optimizers import Delayer

# Test the functions

## Basic Initialization

In [3]:
Ackley(2).__dict__

{'n': 2,
 'domain': (-32.768, 32.768),
 'minimizer': array([0., 0.]),
 'a': 20.0,
 'b': 0.2,
 'c': 6.283185307179586}

In [4]:
Rastrigin(10).__dict__

{'n': 10,
 'domain': (-5.12, 5.12),
 'minimizer': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])}

In [5]:
Rosenbrock(100).__dict__

{'n': 100,
 'domain': (-5.0, 10.0),
 'minimizer': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'a': 1.0,
 'b': 100.0}

In [6]:
Zakharov(1000).__dict__

{'n': 1000,
 'domain': (-5.0, 10.0),
 'minimizer': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.

## Loss Evaluation

In [7]:
import numpy as np

X = np.random.rand(100,10000)
dims = [2, 10, 100, 1000, 10000]

In [8]:
import time
from delay_optimizer.functions.old_functions import (
    ackley_gen,
    rastrigin_gen,
    rosenbrock_gen,
    zakharov_gen
)

def get_old_loss_function(func):
    match str(func).lower():
        case 'ackley':
            return ackley_gen(func.n)
        case 'rastrigin':
            return rastrigin_gen(func.n)
        case 'rosenbrock':
            return rosenbrock_gen(func.n)
        case 'zakharov':
            return zakharov_gen(func.n)

def test_loss(func, x, n):
    f = func(n)
    f_old = get_old_loss_function(f)
    x = x[:,:n]

    t0 = time.time()
    f1 = f.loss(x)
    t1 = time.time()
    f2 = [f.loss(x[i]) for i in range(x.shape[0])]
    t2 = time.time()
    f3 = [f_old(x[i]) for i in range(x.shape[0])]
    t3 = time.time()

    assert np.allclose(f1, f3)
    assert np.allclose(f1, f2)
    assert np.allclose(f2, f3)
    return t1-t0, t2-t1, t3-t2

In [9]:
# Ackley
for d in dims:
    print(*test_loss(Ackley, X, d))

0.0001857280731201172 0.0018610954284667969 0.0018894672393798828
0.0009121894836425781 0.0018253326416015625 0.010658979415893555
0.0004627704620361328 0.002258777618408203 0.002112150192260742
0.005427122116088867 0.007301807403564453 0.004945993423461914
0.038039445877075195 0.027659177780151367 0.026062488555908203


In [10]:
# Rastrigin
for d in dims:
    print(*test_loss(Rastrigin, X, d))

0.00011301040649414062 0.0010266304016113281 0.0009617805480957031
0.0006196498870849609 0.0011293888092041016 0.0010020732879638672
0.0017309188842773438 0.0014655590057373047 0.0014944076538085938
0.003254413604736328 0.0034799575805664062 0.0033440589904785156
0.025737762451171875 0.021487712860107422 0.018725872039794922


In [11]:
# Rosenbrock
for d in dims:
    print(*test_loss(Rosenbrock, X, d))

7.128715515136719e-05 0.0008788108825683594 0.0011162757873535156
0.0002727508544921875 0.0009210109710693359 0.0011589527130126953
0.0011010169982910156 0.0010349750518798828 0.0012493133544921875
0.0015153884887695312 0.0016238689422607422 0.0019321441650390625
0.011744976043701172 0.004252910614013672 0.0045812129974365234


In [12]:
# Zakharov
for d in dims:
    print(*test_loss(Zakharov, X, d))

0.00015473365783691406 0.0061414241790771484 0.005188941955566406
0.0002658367156982422 0.0066127777099609375 0.0019745826721191406
0.0012171268463134766 0.002131938934326172 0.0014562606811523438
0.0003635883331298828 0.0016679763793945312 0.0017631053924560547
0.004600048065185547 0.003712892532348633 0.005046367645263672


## Gradient Evaluation

In [13]:
import numpy as np

X = np.random.rand(100,10000)
dims = [2, 10, 100, 1000, 10000]

In [14]:
import time
from delay_optimizer.functions.old_functions import (
    ackley_deriv_gen,
    rast_deriv_gen,
    rosen_deriv_gen,
    zakharov_deriv_gen
)

def get_old_grad_function(func):
    match str(func).lower():
        case 'ackley':
            return ackley_deriv_gen(func.n)
        case 'rastrigin':
            return rast_deriv_gen(func.n)
        case 'rosenbrock':
            return rosen_deriv_gen(func.n)
        case 'zakharov':
            return zakharov_deriv_gen(func.n)

def test_grad(func, x, n):
    f = func(n)
    f_old = get_old_grad_function(f)
    x = x[:,:n]

    t0 = time.time()
    f1 = f.grad(x)
    t1 = time.time()
    f2 = [f.grad(x[i]) for i in range(x.shape[0])]
    t2 = time.time()
    f3 = [f_old(x[i]) for i in range(x.shape[0])]
    t3 = time.time()

    assert np.allclose(f1, f3)
    assert np.allclose(f1, f2)
    assert np.allclose(f2, f3)
    return t1-t0, t2-t1, t3-t2

In [15]:
# Ackley
for d in dims:
    print(*test_grad(Ackley, X, d))

0.00018334388732910156 0.003925800323486328 0.003774881362915039
0.004918575286865234 0.008687734603881836 0.006947040557861328
0.0012769699096679688 0.0055997371673583984 0.006273508071899414
0.0071866512298583984 0.009026288986206055 0.008199453353881836
0.050015926361083984 0.04297304153442383 0.041222333908081055


In [16]:
# Rastrigin
for d in dims:
    print(*test_grad(Rastrigin, X, d))

7.510185241699219e-05 0.0005922317504882812 0.0004990100860595703
0.00030732154846191406 0.0006930828094482422 0.0005869865417480469
0.0003857612609863281 0.0015645027160644531 0.0008997917175292969
0.0025892257690429688 0.0034923553466796875 0.0025403499603271484
0.02410435676574707 0.02041769027709961 0.019176483154296875


In [17]:
# Rosenbrock
for d in dims:
    print(*test_grad(Rosenbrock, X, d))

0.0001246929168701172 0.005778789520263672 0.006569862365722656
0.0005772113800048828 0.0049724578857421875 0.005286216735839844
0.00026535987854003906 0.0017685890197753906 0.0017848014831542969
0.0024566650390625 0.0026204586029052734 0.0026044845581054688
0.01853799819946289 0.009824514389038086 0.00918269157409668


In [18]:
# Zakharov
for d in dims:
    print(*test_grad(Zakharov, X, d))

0.0001373291015625 0.005235910415649414 0.009942054748535156
0.00042366981506347656 0.0015664100646972656 0.0012929439544677734
0.004384756088256836 0.0016946792602539062 0.0013718605041503906
0.0006747245788574219 0.002216339111328125 0.004332304000854492
0.013399362564086914 0.00696110725402832 0.015995025634765625


# Test the new Optimizers

In [19]:
from delay_optimizer.generators import learning_rate_generator as lr_gen

In [20]:
objective = Ackley(10)
X = np.random.uniform(*objective.domain, size=(25, 10))

In [21]:
import time
from delay_optimizer.optimizers.old_optimizers import (
    GradientDescent as GradientDescentOld,
    Adam as AdamOld,
    Momentum as MomentumOld,
    NesterovMomentum as NesterovMomentumOld,
)

def get_old_optimizer(optimizer):
    match optimizer.__class__.__name__:
        case 'GradientDescent':
            return GradientDescentOld
        case 'Adam':
            return AdamOld
        case 'Momentum':
            return MomentumOld
        case 'NesterovMomentum':
            return NesterovMomentumOld

def test_optimizer(optimizer, objective, X, **kwargs):
    opt = optimizer(**kwargs)
    kwargs['learning_rate'] = kwargs['lr']
    opt_old = get_old_optimizer(opt)(params=kwargs) # Doesn't account for epsilon param

    for x in X:
        opt.initialize(x)
        opt_old.initialize(x)

        for i in range(1000):    # Maxiter=100
            grad = objective.grad(x)

            x1 = opt.step(x, grad)
            x2 = opt_old(x, grad, i+1)
            try:
                assert np.allclose(x1, x2)
            except:
                print("")
                print(x1)
                print(x2)
                print(i)
                raise
            x = x1

In [22]:
test_optimizer(GradientDescent, objective, X, lr=lr_gen.constant(0.01))

In [23]:
test_optimizer(Adam, objective, X, lr=lr_gen.constant(0.01), beta_1=0.9, beta_2=0.999)

In [24]:
test_optimizer(Momentum, objective, X, lr=lr_gen.constant(0.01), gamma=0.8)

In [25]:
test_optimizer(NesterovMomentum, objective, X, lr=lr_gen.constant(0.01), gamma=0.8)

# Test the new Delayer

## Simplified Delayer

In [26]:
from delay_optimizer.generators import learning_rate_generator as lr_gen
from delay_optimizer.generators.delay_distributions import Cyclical

In [27]:
objective = Rastrigin(2)
optimizer = Adam
maxiter = 1000
delay_type = Cyclical([np.array([0,0]), np.array([1,0]), np.array([0,1]), np.array([1,1])], maxiter)
X = np.random.uniform(*objective.domain, size=(100, 2))

params = {'lr': lr_gen.constant(0.01), 'beta_1': 0.9, 'beta_2': 0.999}

In [30]:
from delay_optimizer.optimizers.old_Delayer import Delayer as DelayerOld

def test_delayer(objective, optimizer, delay_type, X, maxiter=1000, **kwargs):
    optimizer = optimizer(**kwargs)
    kwargs['learning_rate'] = kwargs['lr']
    optimizer_old = get_old_optimizer(optimizer)(kwargs)
    delayer = Delayer(objective, optimizer, delay_type)
    delayer_old = DelayerOld(delay_type, objective, optimizer_old)

    for x in X:
        delayer.initialize(x)
        delayer_old.initialize(x)
        D_gen = delay_type.D_gen(objective.n)
        
        for i in range(1, maxiter+1):
            delayer.step()
            x1 = delayer.time_series[0]
            x2 = delayer_old.update(i, next(D_gen))
            try:
                assert np.allclose(x1, x2)
            except:
                print("")
                print(x1)
                print(x2)
                print(i)
                raise
            x = x1


In [31]:
test_delayer(objective, optimizer, delay_type, X, maxiter=maxiter, **params)

## Check Simultaneous Delay Generation for multiple points

In [3]:
from delay_optimizer.generators.delay_distributions import (
    Undelayed, 
    Uniform, 
    Stochastic, 
    Decaying, 
    Partial, 
    Cyclical,
    Constant
)

def test_delay_type(delay_type):
    for size in [2, (2,), (5,2), (3,1,2), (10,5,3,2)]:
        D_gen = delay_type.D_gen(size)
        check_size = size if not isinstance(size, int) else tuple([size])
        for i in range(delay_type.num_delays+10):
            assert next(D_gen).shape == check_size

In [4]:
test_delay_type(Undelayed())

In [5]:
test_delay_type(Uniform(max_L=1, num_delays=10))

In [6]:
test_delay_type(Stochastic(max_L=1, num_delays=10))

In [7]:
test_delay_type(Decaying(max_L=1, num_delays=10))

In [8]:
test_delay_type(Partial(max_L=1, num_delays=10, p=0.5))

In [9]:
D = [np.array([0,0]), np.array([1,0]), np.array([0,1]), np.array([1,1])]
test_delay_type(Cyclical(D, num_delays=10))

In [10]:
D = np.array([1,0])
test_delay_type(Constant(D, num_delays=10))

## Numpy Parallel Delayer

In [1]:
import numpy as np
import time

class Delayer:
    """Class that performs delayed optimization on the given objective function 
    with the given optimizer. The sequence of delays to use is determined by the 
    DelayType object.
    """
    def __init__(self, objective, optimizer, delay_type):
        """Initializer for the Delayer class
            
        Parameters:
            objective (ObjectiveFunction): the loss function to be optimized over, 
                including dimension n, loss function, and gradient function
            optimizer (Optimizer): an initialized class object to perform the
                optimization. Must have a step() method that updates the state
                given state and gradient values.
            delay_type (DelayType): object containing delay parameters
        """
        self.objective = objective
        self.optimizer = optimizer
        self.delay_type = delay_type
        
    def initialize(self, x_init):
        self.optimizer.initialize(x_init)
        self.time_series = np.tile(x_init, (self.delay_type.max_L+1, 1))
        self.delay_generator = self.delay_type.D_gen(self.objective.n)
    
    def step(self):
        """Computes the delayed state and gradient values and takes a step
        with the optimizer.
        """
        # Compute the delayed state and gradient
        D = next(self.delay_generator)              # Get the delay
        del_state = np.diag(self.time_series[D])
        del_grad = self.objective.grad(del_state)
        
        # Roll the time series forward and update
        self.time_series = np.roll(self.time_series, 1, axis=0)
        self.time_series[0] = self.optimizer.step(del_state, del_grad) 

        return self.time_series[0]   
        
    def optimize(self, x_init, maxiter=5000):
        """Computes the time series using the passed optimizer from __init__, 
        saves convergence and time_series which is an array of the states
           
        Parameters:
            x_init (ndarray): the initial state of the calculation
            maxiter (int): the max number of iterations
        Returns:
            (Result): an object containing the optimization data
        """
        self.initialize(x_init) 
        for i in range(maxiter):
            x = self.step()
        return x    


In [ ]:
from delay_optimizer.functions import Ackley
from delay_optimizer.optimizers import Adam

n = 10
objective = Ackley(n)
optimizer = Adam(lr=0.01, beta_1=0.9, beta_2=0.999)
X = np.random.uniform(*objective.domain, size=(100, n))

